## Imports
***

In [1]:
import pandas as pd 
import os
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
print(__version__)# requires version >= 1.9.0
import plotly
import plotly.graph_objs as go
from datetime import datetime

3.3.0


## Helper function
***

Assumed time period is hourly on full hours.

In [2]:
def read_electricity_p(appliances = None, 
                       limit = None, 
                       filename = 'Electricity_P.csv'):
    
    """Reads Electricity_p dataset

    Args:
        appliances: devices to read.
        limit: limit the number of rows to read.
        rel_file_path: relative path to file to read.

    Returns:
        in-memory read pandas dataframe along with the appliances 
    """
    
    # NOTE: data will be provided in google cloud storage
    filename = os.path.dirname(os.path.dirname(os.getcwd()).replace(' ','\ ')) + "/src/model/data/" + filename
    
    # define appliance which shall be displayed
    if appliances is None and limit is None:
        electricity_data = pd.read_csv(filename,sep=",")
    elif appliances is None and limit is not None:
        electricity_data = pd.read_csv(filename,sep=",", nrows = limit)
    elif appliances is not None and limit is None:
        electricity_data = pd.read_csv(filename,sep=",", usecols=(appliances+['UNIX_TS']))
    else:
        electricity_data = pd.read_csv(filename,sep=",",nrows = limit, usecols=(appliances+['UNIX_TS']))

    # conert date column into plotly-interpretable format
    electricity_data['UNIX_TS'] = pd.to_datetime(electricity_data['UNIX_TS'],unit='s').astype(datetime)
    
    # set index to date
    #electricity_data.set_index('UNIX_TS')

    # aggregate on an hour basis
    electricity_data = electricity_data \
        .groupby(electricity_data.UNIX_TS.map(lambda x: x.strftime('%Y-%m-%d %H'))) \
        .mean()

    appliances = electricity_data.columns.values
    return electricity_data, appliances

## Analysis
***

In [11]:
plotly.offline.init_notebook_mode(connected=True)
electricity_data, appliances= read_electricity_p(appliances = ['DNE'])


date = electricity_data.index.values
appliance_consumption = {}
appliance_traces ={}

for appliance in appliances:
    appliance_consumption[appliance] = electricity_data[appliance].values



for appliance in appliances:
    appliance_traces[appliance] = go.Scatter(
                                x = date,
                                y = appliance_consumption[appliance],
                                mode = 'lines',
                                name = 'lines')


for appliance in appliances:
    layout = go.Layout(title = appliance)
    fig = go.Figure(data=[appliance_traces[appliance]], layout=layout)
    plotly.offline.iplot(fig,filename = 'styling_names')



In [5]:
appliances

array(['FGE'], dtype=object)